<h2>Introdução à Ciência de Dados</h2>

Usaremos para nossos estudos futuros a linguagem R. Portanto, primeiramente iremos carregar a biblioteca Tidyverse. Esse ambiente de programação já vem pré-configurado de modo que você não precisa baixá-la, caso resolva executar os seguintes exemplos localmente faça o download. Algumas funções básicas que serão usadas ao longo deste material são:

-   `mean()` Média
-   `median()` Mediana
-   `var()` Variância
-   `sd()` Desvio Padrão

### Sumário:

-   [1. Problema](#cell1)
-   [2. Lendo Dados](#cell2)
-   [3. Entendendo Dados](#cell3)
-   [4. Dplyr para manipulação e transformação de dados](#cell4)

In [ ]:
# Se estiver executando localmente apague o '#'

# install.packages("tidyverse")

In [ ]:
# Carrega tidyverse

library(tidyverse)

Os pacotes que vêm nessa biblioteca e nos auxiliarão na análise de dados são:

-   tidyr: ajuda a criar dados organizados
-   dplyr: manipulação e transformação de dados
-   readr: faz a leitura de arquivos
-   purrr: programação funcional e suas ferramentas
-   ggplot2: vizualização de dados e gráficos.

<a id="cell1"></a>

<h3>Problema</h3>

Antes de colocar a *mão na massa*, vamos primeiro analisar um problema. Digamos que você trabalha para uma empresa nos EUA e irá fazer uma viagem, a trabalho, de Los Angeles a Nova York. É necessário chegar pontualmente ao seu compromisso. Portanto, precisamos saber quais fatores levam ao atraso de um voo, fazer uma estimativa deles e descobrir qual linha aérea é a melhor.

A empresa IBM, mantém em seu site uma série de conjunto de dados. Nesse caso usaremos o <a href="https://developer.ibm.com/exchanges/data/all/airline/">Airline Dataset</a> que contém dados de mais de 200 milhões de voos domésticos feitos em território americano, coletados pelo Departamento de Estatísticas de Transporte dos Estados Unidos. Por questões didáticas usaremos a amostra desses dados e não o arquivo inteiro.

<ul>
    <li>data source: <a href="https://dax-cdn.cdn.appdomain.cloud/dax-airline/1.0.1/lax_to_jfk.tar.gz" target="_blank">https://dax-cdn.cdn.appdomain.cloud/dax-airline/1.0.1/lax_to_jfk.tar.gz</a></li>
    <li>data type: csv</li>
</ul>

<a id="cell2"></a>

<h3>Lendo Dados</h3>

Usamos a função read_csv() para ler arquivos csv, lembrando que ela faz parte do pacote readr. Os seus parâmetros são:

-   `file` (primeiro parâmetro): endereço URL ou caminho no seu computador onde o arquivo se encontra
-   `col_names`: por padrão temos, `col_names = TRUE`. Significa que a primeira linha é o cabeçalho do conjunto de dados.
-   `col_types`: especificamos o tipo de dados em cada coluna. Por padrão a linguagem R *advinha* eles para você. Caso queira, especifique manualmente
    -   Você pode usar os seguinte tipos: `col_logical()`, `col_integer()`, `col_number()`, `col_character()`.

A variável `companhias_aereas` guardará os dados.

In [ ]:
# url onde os dados estão localizados
# url <- "https://dax-cdn.cdn.appdomain.cloud/dax-airline/1.0.1/lax_to_jfk.tar.gz"

# download do arquivo
#download.file(url, destfile = "lax_to_jfk.tar.gz")

# extrair arquivo 
#untar("lax_to_jfk.tar.gz", tar = "internal")

# Lemos o arquivo csv => read_csv (Se for de um formato diferente use a função apropriada, verifique a documentação de readr)
# companhias_aereas <- read_csv("lax_to_jfk/lax_to_jfk.csv",
#                     col_types = cols(
#                      'DivDistance' = col_number(),
#                      'DivArrDelay' = col_number()
#                      ))

# O arquivo já está instalado, portanto leremos direto do diretório dados
companhias_aereas <- read_csv("dados/losAngeles_para_novaYork.csv",
                     col_types = cols(
                      'DivDistance' = col_number(),
                      'DivArrDelay' = col_number()
                      ))

Uma vez lido o conjunto de dados, vamos fazer algumas análises iniciais sobre ele.

-   `head(dataframe, n)`: Retorna as primeiras 6 linhas do conjunto de dados se `n` não for especificado
-   `tail(dataframe, n)`: Retorna as últimas 6 linhas do conjunto de dados se `n` não for especificado
-   `colnames(dataframe)`: Retorna o nome das colunas
-   `dim(dataframe)`: Retorna as dimensões do conjunto de dados.

In [ ]:
# mostra as 3 primeiras linhas
head(companhias_aereas, 3)

In [ ]:
# mostra as 3 últimas linhas
tail(companhias_aereas, 3)

<a id="cell3"></a>

<h3>Entendendo Dados</h3>

Como mencionado anteriormente, estamos interessados em voos feitos de Los Angeles para Nova York, representados respectivamente por LAX e JFK no nosso dataset (usaremos o termo em inglês dataset para nos referimos a conjunto de dados). A sigla JFK faz referência ao Aeroporto Internacional John F. Kennedy. Também queremos focar em informações referentes a possíveis atrasos, as colunas ArrDelay(Atraso na chegada), SecurityDelay (Atraso devido a medidas de segurança), CarrierDelay(atrasos que são causados pela companhia aérea como limpeza do avião), e outras são objetos da nossa análise.

In [ ]:
# A amostra que estamos usando já vem com os voos LAX para JFK separados
# e com as colunas que precisamos para análise
# As dimensões e nomes das colunas do dataset são:

dim(companhias_aereas)
colnames(companhias_aereas)

In [ ]:
# Salvamos o dataset no diretório dados
# Não faremos isso pois já temos o dataset salvo na pasta dados.
# write_csv(companhias_aereas, "dados/losAngeles_para_novaYork.csv")

In [ ]:
#Descobrir o tipo de dados no dataset

sapply(companhias_aereas, typeof) 

<a id="cell4"></a>

<h3> Dplyr para manipulação e transformação de dados </h3>

Algumas funções do pacote dplyr:

-   `select()` Seleciona
-   `filter()` Filtra
-   `summarize()` Resume
-   `arrange()` Organiza linhas de dados por valores das colunas
-   `mutate()` Adiciona novas variáveis
-   `group_by()` Agrupa dados em um novo dataset

In [ ]:
# A média de atrasos na chegada de cada companhia aéra
# Os valores na coluna ArrDelay funcionam da seguinte maneira:
# Se o valor é positivo houve atraso
# Se o valor é negativo ela chegou adiantada ou cedo.
# O operador '>' pode ser lido como então

companhias_aereas %>%
  group_by(Reporting_Airline) %>%
  summarize(atraso_Companhia = mean(ArrDelay, na.rm = TRUE))

# Podemos observar que a empresa com menor tempo médio de atraso na chegada é a AS

In [ ]:
# Um resumo do data_set
glimpse(companhias_aereas)